![alt text](DataKind_orange.png)

# Omidyar Extractives Project 1
## Extract Contract Text (Notebook 1 of 5)
### 1. Extracts contract text using urls to pdf documents contained in contract metadata file downloaded from resourcecontracts.org/contracts
### 2. Appends contract text to original contract metadata file and pickles dataframe for analysis

In [1]:
import urllib
import urllib2
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Progress")

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

In [2]:
contracts = pd.read_csv('contract_data/contract_2017-06-21.csv')
outfile = 'contract_data/contracts_with_text.pkl'

In [4]:
def pdf_from_url_to_txt(url):
    '''
    Takes a url to a pdf document and returns the text as a string
    '''
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Open the url provided as an argument to the function and read the content
    f = urllib2.urlopen(urllib2.Request(url)).read()
    # Cast to StringIO object
    fp = StringIO(f)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=False):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [5]:
def get_contract_text(url):
    '''
    Takes url to a pdf document and returns text as a string
    '''
    txt_file = url[:-3] + 'txt'
    text = urllib.urlopen(txt_file).read()
    if 'AccessDenied' in text[0:1000]:
        text = pdf_from_url_to_txt(url)
    return text

In [6]:
contracts['Contract_Text'] = contracts['Pdf Url'].progress_apply(get_contract_text)

Progress: 100%|██████████| 1496/1496 [42:46<00:00,  1.61s/it]


In [8]:
print "Number of contracts extracted: " + str(len(contracts))

1496


In [7]:
contracts.to_pickle(outfile)